In [24]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianNetwork

# Establishing the structure
model = BayesianNetwork([
    ('Demand', 'Allocation'),
    ('State', 'Allocation'),
    ('Allocation', 'Performance'),
    ('Demand', 'Type'),
    ('Type', 'Volume'),
    ('Performance', 'Need'),
    ('Performance', 'Satisfaction'),
])

# Defining the relationships
Demand_cpd = TabularCPD(
    variable='Demand',
    variable_card=2,
    values=[[.6], [.4]]
)

State_cpd = TabularCPD(
    variable='State',
    variable_card=2,
    values=[[.6], [.4]]
)

Allocation_cpd = TabularCPD(
    variable='Allocation',
    variable_card=2,
    values=[[.7, .8, .95, .8], [.3, .2, .05, .2]],
    evidence=['Demand', 'State'],
    evidence_card=[2, 2]
)

Performance_cpd = TabularCPD(
    variable='Performance',
    variable_card=2,
    values=[[.2, .1], [.8, .9]],
    evidence=['Allocation'],
    evidence_card=[2]
)

Type_cpd = TabularCPD(
    variable='Type',
    variable_card=2,
    values=[[.8, .9], [.2, .1]],
    evidence=['Demand'],
    evidence_card=[2]
)

Volume_cpd = TabularCPD(
    variable='Volume',
    variable_card=2,
    values=[[.9, .95], [.1, .05]],
    evidence=['Type'],
    evidence_card=[2]
)

Satisfaction_cpd = TabularCPD(
    variable='Satisfaction',
    variable_card=2,
    values=[[.9, .95], [.1, .05]],
    evidence=['Performance'],
    evidence_card=[2]
)

Need_cpd = TabularCPD(
    variable='Need',
    variable_card=2,
    values=[[.9, .95], [.1, .05]],
    evidence=['Performance'],
    evidence_card=[2]
)

# Adding the relationships to the model
model.add_cpds(Demand_cpd, State_cpd, Allocation_cpd, Performance_cpd, Satisfaction_cpd, Need_cpd, Type_cpd, Volume_cpd)

# Verifying the structure of the model
model.get_cpds()

[<TabularCPD representing P(Demand:2) at 0x135c7634210>,
 <TabularCPD representing P(State:2) at 0x135c76ecf50>,
 <TabularCPD representing P(Allocation:2 | Demand:2, State:2) at 0x135c76ecc90>,
 <TabularCPD representing P(Performance:2 | Allocation:2) at 0x135c76ece90>,
 <TabularCPD representing P(Satisfaction:2 | Performance:2) at 0x135c76ecc50>,
 <TabularCPD representing P(Need:2 | Performance:2) at 0x135c76ed010>,
 <TabularCPD representing P(Type:2 | Demand:2) at 0x135c76ec350>,
 <TabularCPD representing P(Volume:2 | Type:2) at 0x135c76ed150>]

In [3]:
# Find the paths of the nodes to determine what provides information about a node
print(model.active_trail_nodes('Demand'))
print()

print(model.active_trail_nodes('Type'))
print()

print(model.active_trail_nodes('Satisfaction'))

{'Demand': {'Performance', 'Need', 'Type', 'Satisfaction', 'Volume', 'Demand', 'Allocation'}}

{'Type': {'Performance', 'Need', 'Type', 'Satisfaction', 'Volume', 'Demand', 'Allocation'}}

{'Satisfaction': {'Performance', 'Need', 'Type', 'Demand', 'State', 'Satisfaction', 'Volume', 'Allocation'}}


In [4]:
# Determination of local independencies
print(model.local_independencies('Demand'))
print()

print(model.local_independencies('Volume'))
print()

print(model.local_independencies('Performance'))
print()

model.get_independencies()

(Demand ⟂ State)

(Volume ⟂ State, Performance, Need, Satisfaction, Demand, Allocation | Type)

(Performance ⟂ Demand, Type, State, Volume | Allocation)



(Performance ⟂ Volume | Type)
(Performance ⟂ Type, Volume | Demand)
(Performance ⟂ Demand, Type, State, Volume | Allocation)
(Performance ⟂ Volume | State, Type)
(Performance ⟂ Type, Volume | State, Demand)
(Performance ⟂ Demand, Type, Volume | State, Allocation)
(Performance ⟂ Volume | Need, Type)
(Performance ⟂ Type, Volume | Demand, Need)
(Performance ⟂ Demand, Type, State, Volume | Need, Allocation)
(Performance ⟂ Volume | Satisfaction, Type)
(Performance ⟂ Volume | Demand, Type)
(Performance ⟂ Demand, State, Volume | Allocation, Type)
(Performance ⟂ Type, Volume | Demand, Satisfaction)
(Performance ⟂ Demand, Type, State, Volume | Allocation, Satisfaction)
(Performance ⟂ Type | Demand, Volume)
(Performance ⟂ Demand, Type, State | Allocation, Volume)
(Performance ⟂ State, Type, Volume | Demand, Allocation)
(Performance ⟂ Volume | State, Need, Type)
(Performance ⟂ Type, Volume | State, Need, Demand)
(Performance ⟂ Demand, Type, Volume | State, Need, Allocation)
(Performance ⟂ Volume 

In [25]:
from pgmpy.inference import VariableElimination

# Calculate existing probabilities to verify that everything is correct
inference = VariableElimination(model)
prob_Performance = inference.query(variables=['Performance'])
print(prob_Performance)

+----------------+--------------------+
| Performance    |   phi(Performance) |
+================+====================+
| Performance(0) |             0.1800 |
+----------------+--------------------+
| Performance(1) |             0.8200 |
+----------------+--------------------+


In [6]:
# Perform another probability calculation
prob_Allocation = inference.query(variables=['Allocation'])
print(prob_Allocation)

+---------------+-------------------+
| Allocation    |   phi(Allocation) |
+===============+===================+
| Allocation(0) |            0.8000 |
+---------------+-------------------+
| Allocation(1) |            0.2000 |
+---------------+-------------------+


In [28]:
# Probability of having storage allocation given that the resource state is not free
proba_allocation_state = inference.query(variables=['Allocation'], evidence= {'State':1}) 
print(proba_allocation_state)

+---------------+-------------------+
| Allocation    |   phi(Allocation) |
+===============+===================+
| Allocation(0) |            0.8000 |
+---------------+-------------------+
| Allocation(1) |            0.2000 |
+---------------+-------------------+


In [26]:
# Probability of performance given that allocation is high
proba_allocation_state_demand = inference.query(variables=['Performance'], evidence= {'Allocation':1}) 
print(proba_allocation_state_demand)

+----------------+--------------------+
| Performance    |   phi(Performance) |
+================+====================+
| Performance(0) |             0.1000 |
+----------------+--------------------+
| Performance(1) |             0.9000 |
+----------------+--------------------+
